**ROOM/STATE DATA FORMAT**

(subject to changes?)

Every room/state stored in memory has a label pointing to the beginning of its data block, and variables stored at regular distances relative to that top. 

For all rooms:
- Label + 0: which quality (HP/stats) the room changes (0 for none) [Note: Could this field also be used to change variables, e.g. whether you have a sword?]
- Label + 1: amount of quality change
- Label + 2: type of state (0 for user-input directed, 1 for dice-directed, 2 for variable-directed, 3 for end game, 4 for message)

Then if user-input room: 
- Label + 3, 4, 5: addresses of destinations for 1, 2, and 3 inputs
- Label + 6: Description string in compressed format

Or if dice room: 
- Label + 3: Stat to check
- Label + 4: Difficulty of check
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if variable room:
- Label + 3: Address of variable to check 
- Label + 4: Value of that variable that counts as a success
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if end room: 
- Label + 3: Endgame message

Or if simple message room: 
- Label + 3: Unconditional destination
- Label + 4: Message to display

In [ ]:
;; Room templates

ROOM_UINPUT:   
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL A2 ;; Choice 1
    .FILL A3 ;; Choice 2
    .FILL END ;; Choice 3
    .STRINGC "Room text"
    
ROOM_DICE: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL A5 ;; Destination if success
    .FILL A3 ;; Destination if failure
    
ROOM_VARIABLE:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL SWD ;; VAR_ADDRESS ;; What variable to check
    .FILL #1 ;; VALUE       ;; What value counts as a success
    .FILL ROOM_UINPUT ;; Destination if success
    .FILL ROOM_RANDOM ;; Destination if failure
    
ROOM_ENDING: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "Ending text"
    
MESSAGE_RM: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL ROOM_UINPUT ;; Unconditional destination
    .STRINGC "Message text"

In [ ]:
;; A Rooms
A1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL A2 ;; Choice 1
    .FILL A3 ;; Choice 2
    .FILL H1 ;; Choice 3
    .STRINGC "You are in your room in Brecon! Your paper is written, your teeth are brushed, and your bag is packed. All you need to do is eat lunch, print your paper, and get to class on time to turn it in. As you're leaving your room you see a fencer, notorious for being chatty. Would you like to:\n 1) Avoid them (Wisdom) \n 2) Talk to them (Charisma)\n 3) Go back to sleep\n"
    
A2: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on user input
    .FILL #4 ;; Stat for dice roll: Wisdom
    .FILL #8 ;; Difficulty of check
    .FILL A5 ;; Destination if success
    .FILL A3 ;; Destination if failure
    
A3: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL A6 ;; Destination if success
    .FILL A4 ;; Destination if failure
    
A4:
    .FILL #1 ;; Quality changed: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL B1 ;; Unconditional destination
    .STRINGC "Unfortunately, you got caught up in conversation and lost track of time. Now you're running late! You hurry towards the Valley of Death.\n"
    
A5:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL B1 ;; Unconditional destination
    .STRINGC "You successfully avoided what could have been a long conversation! You head towards the Valley of Death, ahead of schedule.\n"
    
A6:
    .FILL #5 ;; Quality changed: Sword
    .FILL #1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL A7 ;; Unconditional destination
    .STRINGC "You got a little caught up in conversation, but the fencer gave you this cool sword!\n"
    
A7: 
    .FILL #1 ;; Quality changed: HP
    .FILL #1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL B1 ;; Unconditional destination
    .STRINGC "That's gotta be worth being behind schedule, right? You hurry towards the Valley of Death.\n"

In [ ]:
;; B Rooms

B1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL B2 ;; Choice 1
    .FILL B4 ;; Choice 2
    .FILL B5 ;; Choice 3
    .STRINGC "You've reached the Valley of Death! Its depths loom before you. Which way would you like to go?\n 1) You don't have time for lunch -- just go behind Rhoads and go straight to Canaday to print your paper.\n 2) Brave the stairs -- it's the fastest way to lunch. (Vitality)\n 3) Go around through Park -- it may be slower, but at least you don't have to climb as many stairs.\n"
    
B2:
    .FILL #1 ;; Quality changed: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL B3 ;; Unconditional destination
    .STRINGC "Going behing Rhoads means you have plenty of time to get that paper ready, but...(+HP)\n"
    
B3:
    .FILL #3 ;; Quality changed: Vitality
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL C1 ;; Unconditional destination
    .STRINGC "...you're starting to regret skipping lunch... Ah well, too late now. You need to print your paper! (-Vitality)\n"
    
B4:
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #3 ;; Stat for dice roll: Vitality
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL B7 ;; Destination if success
    .FILL B6 ;; Destination if failure
    
B5: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL D1 ;; Unconditional destination
    .STRINGC "Forget stairs -- you go through Park and take the elevator. Much better! But the detour costs you some time. You'd better hurry if you want to make it to lunch!\n"
    
B6: .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL D1 ;; Unconditional destination
    .STRINGC "Whew! That was a slog, but you made it to the top of the stairs. On to lunch!\n"
    
B7: .FILL #1 ;; Quality changed: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL D1 ;; Unconditional destination
    .STRINGC "The stairs got you this time! You made it to the top, but it took a little longer than you might have liked. You'd better hurry if you want to make it lunch!\n"



In [ ]:
;; C Rooms

C1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL C1 ;; Choice 1
    .FILL C2 ;; Choice 2
    .FILL C4 ;; Choice 3
    .STRINGC "You enter Canaday and head for the printers. But disaster strikes -- there's only one open printer, and it's out of toner! What do you do?\n 1) Talk to the Help Desk \n 2) Talk to the circulation desk \n 3) Talk to the person using the other printer. Maybe they'd be willing to pause the textbook they're printing so you can squeeze out a few pages... (Charisma)\n"
    
C2: ;;PUZZLE???

C3: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL C5 ;; Unconditional destination
    .STRINGC "You did not manage to find the book, and are forced to wait until the person at the next printer has finished printing out their whole textbook before you can print your paper. Now you're really running late -- you'd better book it!\n"
    
C4: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL C5 ;; Destination if success
    .FILL C6 ;; Destination if failure

C5: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL E1 ;; Unconditional destination
    .STRINGC "You've printed out your paper and left Canaday! Now all you have to do is make it to class on time -- no sweat!\n"
    
C6:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL C5 ;; Unconditional destination
    .STRINGC "You couldn't even convince them to let you print out a few measly pages, and have to wait for them to finish printing. Now you're really running late -- you'd better book it!\n"
    


In [ ]:
;; D Rooms

D1: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL D2 ;; Choice 1
    .FILL D3 ;; Choice 2
    .FILL D4 ;; Choice 3
    .STRINGC "You reach Erdman just as the lunch rush is starting. What are you going to do first?\n 1) Try to find a seat (Wisdom) \n 2) Look for a friend (Charisma) \n 3) Get food\n"
    
D2: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #4 ;; Stat for dice roll: Wisdom
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL D11 ;; Destination if success
    .FILL D6 ;; Destination if failure
    
D3: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #1 ;; 1: Based on dice roll
    .FILL #2 ;; Stat for dice roll: Charisma
    ;; 1 HP, 2 CHA, 3 VIT, 4 WIS
    .FILL #8 ;; Difficulty of check
    .FILL D12 ;; Destination if success
    .FILL D6 ;; Destination if failure
    
D4: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL D7 ;; Choice 1
    .FILL D8 ;; Choice 2
    .FILL D13 ;; Choice 3
    .STRINGC "You go up to hot line but they've just run out of mac & cheese. It's going to be a little while before they put out more, but you really had your heart set on that cheesy goodness. What will you do?\n 1) Wait for more mac & cheese \n 2) Just grab something and go \n 3) Forget it, who needs lunch anyway? Just go print your paper. \n"
    
D5:
    .FILL #3 ;; Quality changed: Vitality
    .FILL #+1 ;; Amount of change: +1
    .FILL #4 ;; 4: Message state
    .FILL D9 ;; Unconditional destination
    .STRINGC "You leave Erdman, full and refreshed! +Vitality	You leave Erdman, full and refreshed! +Vitality\n"
    
D6: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-1 ;; Amount of change: -1
    .FILL #4 ;; 4: Message state
    .FILL D5 ;; Unconditional destination
    .STRINGC "It was too crowded to find anything -- after wasting some time looking, you just grab some food and get out. -HP\n"
    
D7: 
